In [1]:
import datasets
import os

In [2]:
data = datasets.load_dataset("jiajinda/bookcorpusopen")

In [3]:
data

DatasetDict({
    train: Dataset({
        features: ['title', 'text'],
        num_rows: 17868
    })
})

In [4]:
data = data['train']

In [5]:
lis = [int(i * data.num_rows / 10) for i in range(1, 10)]

In [6]:
lis.append(data.num_rows)

In [7]:
lis

[1786, 3573, 5360, 7147, 8934, 10720, 12507, 14294, 16081, 17868]

In [8]:
def corpa_cleaning(data):
    START_LEN = 50
    MIN_LEN = 10
    def logic(p):
        nonlocal keep
        if keep:
            return len(data) > MIN_LEN
        elif len(p.strip()) >= START_LEN:
            keep = True
            return True
        return False
    new_Corpa = []
    for text in data['text']:
        keep = False
        para = text.split("\n\n")
        cleaned_para = [p.strip() for p in para if logic(p)]
        if cleaned_para:
            res = " ".join(cleaned_para)
            res = res.replace('\n',' ')
            new_Corpa.append(res)
        else:
            new_Corpa.append("")
    return {"cleaned_text": new_Corpa}

In [9]:
new_data = data.map(corpa_cleaning, remove_columns=['text'], num_proc=os.cpu_count(),load_from_cache_file=False, batched=True)

Map (num_proc=24):   0%|          | 0/17868 [00:00<?, ? examples/s]

In [10]:
# val1 = 0
# for i, val2 in enumerate(lis):
#     selectedData = data.select(range(val1,val2))
#     new_data = selectedData.map(corpa_cleaning, remove_columns=['text'], num_proc=os.cpu_count(),load_from_cache_file=False, batched=True)
#     print(i)
#     new_data.save_to_disk(f"cleaned_text{i}")
#     val1 = val2

In [11]:
new_data.save_to_disk(f"cleaned_text")

Saving the dataset (0/1 shards):   0%|          | 0/17868 [00:00<?, ? examples/s]

In [12]:
x = datasets.load_from_disk("cleaned_text")

In [13]:
x

Dataset({
    features: ['title', 'cleaned_text'],
    num_rows: 17868
})

In [14]:
x[3]['cleaned_text']

"License Notes: This ebook is licensed for your personal enjoyment only. This ebook may not be re-sold or given away to other people. If you would like to share this ebook with another person, please purchase an additional copy for each person you share it with. If you're reading this book and did not purchase it, or it was not purchased for your use only, then you should return to Smashwords.com and purchase your own copy. Thank you for respecting the hard work of this author."

In [15]:
print(new_data[1]["cleaned_text"])

This ebook is licensed for your personal enjoyment only. This ebook may not be re-sold or given away to other people. If you would like to share this book with another person, please purchase an additional copy for each person you share it with. If you're reading this book and did not purchase it, or it was not purchased for your use only, then you should return to Smashwords.com and purchase your own copy. Thank you for respecting the hard work of this author.
